# Classification multi etiquette par exctraction de features

In [15]:
# Chargement des modules necessaire :

import os

import random
from matplotlib import pyplot as plt

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Model

In [16]:
# On declare les chemins vers les donnees :

Image = 'Data'
        
train_data_dir = 'Data/TRAIN'
validation_data_dir = 'Data/VALIDATION'
test_data_dir = 'Data/TEST'

# Dimmension et path :
img_width, img_height = 224, 224

#ImageTRAINCOVID = os.listdir(Image + '/TRAIN/COVID')
ImageTRAINATELECTASIS = os.listdir(Image + '/TRAIN/ATELECTASIS')
ImageTRAINNORMAL = os.listdir(Image + '/TRAIN/NORMAL')
ImageTRAINEFFUSION = os.listdir(Image + '/TRAIN/EFFUSION')
ImageTRAINEMPHYSEMA = os.listdir(Image + '/TRAIN/EMPHYSEMA')
ImageTRAININFILTRATION = os.listdir(Image + '/TRAIN/INFILTRATION')
ImageTRAINPNEUMONIE = os.listdir(Image + '/TRAIN/PNEUMONIE')

#ImageVALIDATIONCOVID = os.listdir(Image + '/VALIDATION/COVID')
ImageVALIDATIONATELECTASIS = os.listdir(Image + '/VALIDATION/ATELECTASIS')
ImageVALIDATIONNORMAL = os.listdir(Image + '/VALIDATION/NORMAL')
ImageVALIDATIONEFFUSION = os.listdir(Image + '/VALIDATION/EFFUSION')
ImageVALIDATIONEMPHYSEMA = os.listdir(Image + '/VALIDATION/EMPHYSEMA')
ImageVALIDATIONINFILTRATION = os.listdir(Image + '/VALIDATION/INFILTRATION')
ImageVALIDATIONPNEUMONIE = os.listdir(Image + '/VALIDATION/PNEUMONIE')

#ImageTESTCOVID = os.listdir(Image + '/TEST/COVID')
ImageTESTATELECTASIS = os.listdir(Image + '/TEST/ATELECTASIS')
ImageTESTNORMAL = os.listdir(Image + '/TEST/NORMAL')
ImageTESTEFFUSION = os.listdir(Image + '/TEST/EFFUSION')
ImageTESTEMPHYSEMA = os.listdir(Image + '/TEST/EMPHYSEMA')
ImageTESTINFILTRATION = os.listdir(Image + '/TEST/INFILTRATION')
ImageTESTPNEUMONIE = os.listdir(Image + '/TEST/PNEUMONIE')

#print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINATELECTASIS)) + ' images d\'entrainement de patient atelectasis.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient sain.') 
print('Il y a ' + str(len(ImageTRAINEFFUSION)) + ' images d\'entrainement de patient effusion.') 
print('Il y a ' + str(len(ImageTRAINEMPHYSEMA)) + ' images d\'entrainement de patient emphyseme.') 
print('Il y a ' + str(len(ImageTRAININFILTRATION)) + ' images d\'entrainement de patient infltration.')
print('Il y a ' + str(len(ImageTRAINPNEUMONIE)) + ' images d\'entrainement de patient pneumonie.')

#print('Il y a ' + str(len(ImageVALIDATIONCOVID)) + ' images de validation de patient covid.') 
print('Il y a ' + str(len(ImageVALIDATIONATELECTASIS)) + ' images de validation de patient atelectasis.') 
print('Il y a ' + str(len(ImageVALIDATIONNORMAL)) + ' images de validation de patient non-covid.') 
print('Il y a ' + str(len(ImageVALIDATIONEFFUSION)) + ' images de validation de patient effusion.') 
print('Il y a ' + str(len(ImageVALIDATIONEMPHYSEMA)) + ' images de validation de patient emphyseme.') 
print('Il y a ' + str(len(ImageVALIDATIONINFILTRATION)) + ' images de validation de patient infltration.')
print('Il y a ' + str(len(ImageVALIDATIONPNEUMONIE)) + ' images de validation de patient pneumonie.')

#print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTATELECTASIS)) + ' images test de patient atelectasis.') 
print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images test de patient non-covid.')
print('Il y a ' + str(len(ImageTESTEFFUSION)) + ' images test de patient effusion.') 
print('Il y a ' + str(len(ImageTESTEMPHYSEMA)) + ' images test de patient emphyseme.') 
print('Il y a ' + str(len(ImageTESTINFILTRATION)) + ' images test de patient infltration.')
print('Il y a ' + str(len(ImageTESTPNEUMONIE)) + ' images test de patient pneumonie.')

Il y a 800 images d'entrainement de patient atelectasis.
Il y a 1106 images d'entrainement de patient sain.
Il y a 800 images d'entrainement de patient effusion.
Il y a 1380 images d'entrainement de patient emphyseme.
Il y a 1348 images d'entrainement de patient infltration.
Il y a 1000 images d'entrainement de patient pneumonie.
Il y a 102 images de validation de patient atelectasis.
Il y a 930 images de validation de patient non-covid.
Il y a 133 images de validation de patient effusion.
Il y a 193 images de validation de patient emphyseme.
Il y a 193 images de validation de patient infltration.
Il y a 333 images de validation de patient pneumonie.
Il y a 102 images test de patient atelectasis.
Il y a 872 images test de patient non-covid.
Il y a 133 images test de patient effusion.
Il y a 193 images test de patient emphyseme.
Il y a 387 images test de patient infltration.
Il y a 333 images test de patient pneumonie.


In [17]:
# Preprocessing :

# On rescale les images :

datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :

batch_size = 32 

# On prepare les tableaux de donnees depuis les images :

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

Found 7816 images belonging to 7 classes.
Found 2102 images belonging to 7 classes.
Found 2260 images belonging to 7 classes.


In [18]:
# Chargement de DENSE121 sans la partie fully-connected avec le reseau convolutif :

model_vgg = applications.VGG16(include_top=False, weights='imagenet')

In [ ]:
# On utilse le model VGG16 pour extraire les features de nos images 
# (on fait recupere la sortie du reseau convolutionnel) :
train_features = model_vgg.predict_generator(train_generator, 7816 // batch_size)

validation_features = model_vgg.predict_generator(validation_generator, 2102 // batch_size)

# L'opération étant longue on enregistre les features obtenus :
np.save(open('models/trainFeatures_VGG16_AllDataExtend.npy', 'wb'), train_features) # ecriture en binaire necessaire
np.save(open('models/validationFeatures_VGG16_AllDataExtend.npy', 'wb'), validation_features) # Idem

In [10]:
# Si l'operation à deja été effectuer on charge les features :

train_features = np.load(open('models/trainFeatures_VGG16_AllData.npy', 'rb'))
validation_features = np.load(open('models/validationFeatures_VGG16_AllData.npy', 'rb'))

In [11]:
train_labels = np.array([0] * 800 + [1] * 1386 + [2] * 800 + [3] * 1380 + [4] * 1348 + [5] * 1102 + [6] * 992 )

validation_labels = np.array([0] * 102 + [1] * 219 + [2] * 128 + [3] * 192 + [4] * 192 + [5] * 928 + [6] * 319 )

In [12]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_features.shape[1:]))
model_top.add(Dense(64, activation='relu'))
model_top.add(Dense(7, activation='softmax'))

# On compile :
model_top.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# On affiche :
model_top.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1605696   
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
Total params: 1,606,151
Trainable params: 1,606,151
Non-trainable params: 0
_________________________________________________________________


In [13]:
epochs = 20

batch_size = 32 

In [14]:
history = model_top.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

ValueError: Input arrays should have the same number of samples as target arrays. Found 3520 input samples and 7808 target samples.

In [ ]:
model_top.save('models/VGG16_Full_MAXG_224_224')

In [ ]:
epochs = 20

history2 = model_top.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

In [ ]:
epochs = 20

history3 = model_top.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

In [ ]:
model_top.save('models/VGG16_Full_MAX2G_224_224')